In [ ]:
import pandas as pd
import numpy as np
import sys
import nltk
import sklearn
import pandas
import numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dff = pd.read_csv('/content/drive/MyDrive/DATA /total.csv')

dff

,Position,Description,COVID
0,Nature Club Leader (Nanaimo),NatureKids is designed to get children aged 5-...,No
1,Nature club co-leader (NatureKids Cowichan Val...,NatureKids is designed to get children aged 5-...,No
2,Treasurer,Treasurer\nThe Board of the Friends of the Van...,Yes
3,Distribution Volunteer,"Each week we assist 28,000 people to access fo...",No
4,Warehouse Volunteers,The Greater Vancouver Food Bank has distributi...,No
...,...,...,...
1033,Letters from home have tremendous impact for o...,Would you like to en...,Yes
1034,Support Female Caregivers Virtually-Soldiers' ...,www....,Yes
1035,Support Female Caregivers Virtually-Soldiers' ...,The Women of V...,Yes
1036,"Sewing, crafting, crocheting - Virtual Opportu...",Provide loving...,Yes


In [ ]:
dff.groupby(by="COVID").count()

,Position,Description
COVID,,
No,402,402
U,435,435
Yes,201,201


In [ ]:
dff2=dff[dff.COVID!='U']
desc=dff2.iloc[:,1]
desc

0       NatureKids is designed to get children aged 5-...
1       NatureKids is designed to get children aged 5-...
2       Treasurer\nThe Board of the Friends of the Van...
3       Each week we assist 28,000 people to access fo...
4       The Greater Vancouver Food Bank has distributi...
                              ...                        
1033                              Would you like to en...
1034                                              www....
1035                                    The Women of V...
1036                                    Provide loving...
1037                              Provide lovingly cre...
Name: Description, Length: 603, dtype: object

In [ ]:
classes = dff2.iloc[:,2]
print(classes.value_counts())



No     402
Yes    201
Name: COVID, dtype: int64


In [ ]:
# convert class labels to binary values, 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(Y[:10])

[0 0 1 0 0 0 1 1 1 1]


In [ ]:

processed = desc.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')
processed = processed.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'email')
# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')



# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

processed = processed.str.lower()

In [ ]:
!pip install nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))



In [ ]:


# Remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))



In [ ]:
# for training 
from nltk.tokenize import word_tokenize

# create bag-of-words
all_words = []

for a in processed:
    words = word_tokenize(a)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)


In [ ]:


# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]



In [ ]:
# The find_features function will determine which of the 1500 word features are contained in the review
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Lets see an example!
features = find_features(processed[0])
#for key, value in features.items():
   # if value == True:
     #   print( key)


In [ ]:
# Now lets do it for all
desc1 = zip(processed , Y)
# define a seed for reproducibility
seed = 1
np.random.seed = seed
#np.random.shuffle(desc1)

# call find_features function for each description
featuresets = [(find_features(text), label) for (text, label) in desc1]




In [ ]:
# we can split the featuresets into training and testing datasets using sklearn
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [ ]:


# We can use sklearn algorithms in NLTK
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'rbf'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))



SVC Accuracy: 75.49668874172185


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(alpha=0.05),
    SVC(kernel = 'rbf')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 72.18543046357617
Decision Tree Accuracy: 76.82119205298014
Random Forest Accuracy: 76.82119205298014
Logistic Regression Accuracy: 77.48344370860927
SGD Classifier Accuracy: 78.1456953642384
Naive Bayes Accuracy: 85.43046357615894
SVM Linear Accuracy: 75.49668874172185


In [ ]:
# Define models to train
names = [
         "Naive Bayes"]

classifiers = [
    
    MultinomialNB(alpha=0.05),

]

models = zip(names, classifiers)

for name, model in models:
    nltk_model2 = SklearnClassifier(model)
    nltk_model2.train(training)
    accuracy = nltk.classify.accuracy(nltk_model2, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

Naive Bayes Accuracy: 85.43046357615894


In [ ]:
txt_features, labels = zip(*testing)

prediction = nltk_model2.classify_many(txt_features)

print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['yes', 'No']],
    columns = [['predicted', 'predicted'], ['yes', 'No']])

              precision    recall  f1-score   support

           0       0.84      0.95      0.89        92
           1       0.89      0.71      0.79        59

    accuracy                           0.85       151
   macro avg       0.87      0.83      0.84       151
weighted avg       0.86      0.85      0.85       151



predicted    
                 yes  No
actual yes        87   5
       No         17  42

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

cm =[[0,0],[0,0]]
conf_matrix =[[0,0],[0,0]]
j=0
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(featuresets):
    
    training1 = [featuresets[int(i)] for i in train_index] 
    testing1 = [featuresets[int(i)] for i in test_index]
    names = [
            "Naive Bayes"]

    classifiers = [
        
        MultinomialNB(0.05),

    ]
    models2 = zip(names, classifiers)

    
    for name, model in models2:
        nltk_model4 = SklearnClassifier(model)
        nltk_model4.train(training1)
        accuracy = nltk.classify.accuracy(nltk_model4, testing1)*100
        #print("{} Accuracy: {}".format(name, accuracy))
    txt_features, labels = zip(*testing1)

    prediction = nltk_model.classify_many(txt_features)


    cm = confusion_matrix(labels, prediction)
    j+=1
    conf_matrix=conf_matrix+cm

    
print("cross_validation confusion_matrix is =\n", conf_matrix)



cross_validation confusion_matrix is =
 [[401   1]
 [ 56 145]]


In [ ]:
(conf_matrix[0,0]+conf_matrix[1,1])/conf_matrix.sum()

0.9054726368159204

Test file



In [ ]:
dd= pd.read_csv("/content/drive/MyDrive/DATA /Vw_whole.csv")
desc2=dd["merge"]
testing = zip(desc2 ,np.zeros(np.shape(desc2)[0]) )
testing=[(find_features(text), label) for (text, label) in testing]
training= featuresets

In [ ]:

from nltk.classify.scikitlearn import SklearnClassifier

names = ["Naive Bayes"]
classifiers = [MultinomialNB(0.05)]
models2 = zip(names, classifiers)  
for name, model in models2:
      nltk_model3 = SklearnClassifier(model)
      nltk_model3.train(training) 
      txt_features, labels = zip(*testing)
      prediction = nltk_model3.classify_many(txt_features)




In [ ]:
dd["label"]=prediction
dd


,merge,1,10,11,12,13,14,15,16,17,2,3,4,5,6,7,8,9,label
0,microfin busi develop volunt help process busi...,1,0,1,1,0,0,0,1,1,0,1,1,1,0,0,1,1,0
1,microfin econom empower particip micro financ ...,1,1,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0
2,even coronavirus diseas covid pandem progress ...,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,share world popul live extrem poverti declin p...,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,share world worker live extrem poverti fell ha...,1,0,1,1,1,0,0,1,1,0,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,support implement sdgs steadi fragil major per...,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1
1237,net oda flow member countri develop assist com...,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1
1238,remitt flow low middl incom countri estim reac...,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
1239,global merchandis trade project declin per cen...,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1


In [ ]:
dd.sum(axis=0)

merge    microfin busi develop volunt help process busi...
1                                                     1099
10                                                     173
11                                                    1138
12                                                     786
13                                                     654
14                                                     384
15                                                     818
16                                                     807
17                                                     424
2                                                      818
3                                                     1128
4                                                     1129
5                                                      498
6                                                      720
7                                                      151
8                                                     11